In [0]:
import os
from io import StringIO
import pandas as pd
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import time
from datetime import datetime

In [0]:
    #    os.makedirs("/Volumes/workspace/default/myprojectdata/raw_chunks/", exist_ok=True)
    #    os.makedirs("/Volumes/workspace/default/myprojectdata/output_bucket/", exist_ok=True)
    #    os.makedirs("/Volumes/workspace/default/myprojectdata/processed_file/", exist_ok=True)

In [0]:
display(dbutils.fs.ls("/Volumes/workspace/default/myprojectdata/raw_chunks/"))

path,name,size,modificationTime
dbfs:/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_0.csv,chunk_0.csv,798072,1749896974000
dbfs:/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_1.csv,chunk_1.csv,798789,1749896976000
dbfs:/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_10.csv,chunk_10.csv,809404,1749896986000
dbfs:/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_11.csv,chunk_11.csv,809646,1749896987000
dbfs:/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_12.csv,chunk_12.csv,809769,1749896988000
dbfs:/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_13.csv,chunk_13.csv,809803,1749896990000
dbfs:/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_14.csv,chunk_14.csv,810058,1749896991000
dbfs:/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_15.csv,chunk_15.csv,809107,1749896992000
dbfs:/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_16.csv,chunk_16.csv,187728,1749896993000
dbfs:/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_2.csv,chunk_2.csv,804491,1749896977000


In [0]:
def ingest_transactions():
    """
    Ingests transactions from a CSV file and splits them into smaller chunks.

    This function reads a large transactions CSV file from a specified path, splits it into smaller chunks of 10,000 rows each,
    and saves each chunk as a separate CSV file in a specified directory. It waits for 1 second between processing each chunk.

    The function performs the following steps:
    1. Reads the full transactions file from the specified path.
    2. Splits the data into chunks of 10,000 rows each.
    3. Converts each chunk to a CSV string.
    4. Saves each chunk locally in the specified directory.
    5. Waits for 1 second before processing the next chunk.

    Returns:
        None
    """
    transactions_path = "/Volumes/workspace/default/myprojectdata/transactions.csv"
    full_df = pd.read_csv(transactions_path)
    
    chunk_size = 10000
    total_chunks = len(full_df) // chunk_size + 1
    
    for i in range(total_chunks):
        start_idx = i * chunk_size
        end_idx = (i + 1) * chunk_size
        chunk = full_df.iloc[start_idx:end_idx]
        
        csv_buffer = StringIO()
        chunk.to_csv(csv_buffer, index=False)
        
        local_path = f"/Volumes/workspace/default/myprojectdata/raw_chunks/chunk_{i}.csv"
        with open(local_path, 'w') as f:
            f.write(csv_buffer.getvalue())
        
        time.sleep(1)

if __name__ == "__main__":
    ingest_transactions()

In [0]:
def process_pattern1(df):
    """
    Detects pattern 1 in the given DataFrame.

    This function identifies the top 1% transactions by amount for each merchant, 
    joins them with customer importance data, filters for transactions with weight 
    less than or equal to 0.01, and further filters for merchants with more than 
    50,000 transactions. The final detections are saved.

    Args:
        df (DataFrame): Input DataFrame containing transaction data.

    Returns:
        None
    """
    window = Window.partitionBy("merchant").orderBy(col("amount").desc())
    ranked_df = df.withColumn("percent_rank", percent_rank().over(window))
    
    top_1_percent = ranked_df.filter(col("percent_rank") <= 0.01)
    
    customer_importance = spark.read.csv("/Volumes/workspace/default/myprojectdata/CustomerImportance.csv", header=True)
    joined_df = top_1_percent.join(customer_importance, 
                                 (top_1_percent.customer == customer_importance.Source) &
                                 (top_1_percent.merchant == customer_importance.Target))
    
    bottom_1_weight = joined_df.filter(col("weight") <= 0.01)
    
    merchant_totals = df.groupBy("merchant").agg(count("*").alias("totalTransactions"))
    valid_merchants = merchant_totals.filter(col("totalTransactions") > 50000)
    
    final_detections = bottom_1_weight.join(valid_merchants, "merchant")
    
    output = final_detections.select(
        lit(datetime.now()).alias("YStartTime"),
        lit(datetime.now()).alias("detectionTime"),
        lit("PatId1").alias("patternId"),
        lit("UPGRADE").alias("ActionType"),
        col("customer"),
        col("merchant").alias("MerchantId")
    )
    
    save_detections(output)

def process_pattern2(df):
    """
    Detects pattern 2 in the given DataFrame.

    This function calculates average transaction amount and transaction count 
    for each customer-merchant pair, filters for pairs with average amount less 
    than 23 and transaction count greater than or equal to 80, and joins with 
    customer importance data. The final detections are saved.

    Args:
        df (DataFrame): Input DataFrame containing transaction data.

    Returns:
        None
    """
    customer_stats = df.groupBy("customer", "merchant").agg(
        avg("amount").alias("avgAmount"),
        count("*").alias("transactionCount")
    )
    
    pattern2_detections = customer_stats.filter(
        (col("avgAmount") < 23) & 
        (col("transactionCount") >= 80))
    
    customers = spark.read.csv("/Volumes/workspace/default/myprojectdata/CustomerImportance.csv", header=True)
    final_detections = pattern2_detections.join(customers, (pattern2_detections.customer == customers.Source))
    
    output = final_detections.select(
        lit(datetime.now()).alias("YStartTime"),
        lit(datetime.now()).alias("detectionTime"),
        lit("PatId2").alias("patternId"),
        lit("CHILD").alias("ActionType"),
        col("customer"),
        col("merchant").alias("Merchant")
    )
    
    save_detections(output)

def process_pattern3(df):
    """
    Detects pattern 3 in the given DataFrame.

    This function calculates the distinct count of customers by gender for each 
    merchant, pivots the data to have separate columns for male and female counts, 
    and filters for merchants with more male customers than female customers. 
    The final detections are saved.

    Args:
        df (DataFrame): Input DataFrame containing transaction data.

    Returns:
        None
    """
    gender_counts = df.groupBy("merchant", "gender").agg(countDistinct("customer").alias("count"))
    pivoted = gender_counts.groupBy("merchant").pivot("gender", ["Male", "Female"]).sum("count")
    pattern3_detections = pivoted.filter(col("Male") > col("Female"))
    
    output = pattern3_detections.select(
        lit(datetime.now()).alias("YStartTime"),
        lit(datetime.now()).alias("detectionTime"),
        lit("PatId3").alias("patternId"),
        lit("DEI-NEEDED").alias("ActionType"),
        lit("").alias("customerName"),
        col("merchant").alias("MerchantId")
    )
    
    save_detections(output)

In [0]:
def save_detections(df):
    """
    Saves detection results in batches of 50 rows.

    This function saves the detection results DataFrame to CSV files in batches of 
    50 rows each. The files are named with a timestamp and batch index.

    Args:
        df (DataFrame): DataFrame containing detection results.

    Returns:
        None
    """
    count = df.count()
    for i in range(0, count, 50):
        batch = df.limit(50).offset(i)
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        output_path = f"/Volumes/workspace/default/myprojectdata/output_bucket/detections_{timestamp}_{i}"
        batch.write.csv(output_path, mode="overwrite", header=True)

In [0]:
def detect_patterns():
    """
    Detects patterns in transaction data chunks.

    This function continuously monitors a directory for new transaction data chunks,
    processes each chunk to detect patterns, and moves the processed chunk to a 
    different directory. It performs the following steps:
    1. Lists all CSV files in the specified local directory.
    2. For each file, checks if it has already been processed.
    3. Reads the file into a Spark DataFrame.
    4. Applies pattern detection functions to the DataFrame.
    5. Moves the processed file to the processed directory.
    6. Waits for 1 second before checking for new files.

    Returns:
        None
    """
    local_path = "/Volumes/workspace/default/myprojectdata/raw_chunks/"
    processed_path = "/Volumes/workspace/default/myprojectdata/processed_file/"
    
    while True:
        chunks = [f for f in os.listdir(local_path) if f.endswith(".csv")]
        
        for file_name in chunks:
            file_path = os.path.join(local_path, file_name)
            processed_file_path = os.path.join(processed_path, file_name)
            
            if os.path.exists(processed_file_path):
                continue
            
            df = spark.read.csv(file_path, header=True, inferSchema=True)
            
            process_pattern1(df)
            process_pattern2(df)
            process_pattern3(df)
            
            os.rename(file_path, processed_file_path)
            
        time.sleep(1)


if __name__ == "__main__":
    detect_patterns()